In [21]:
# Data Manipulation
import pandas as pd
from numpy import *
from datetime import timedelta
import yfinance as yf
from tabulate import tabulate

# Math & Optimization
from scipy.stats import norm
from scipy.optimize import fsolve

# Plotting
import matplotlib.pyplot as plt
import cufflinks as cf
cf.set_config_file(offline=True)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
def bs_price_cal(S, K, t, sigma_imp, u, cp):
    d1 = (np.log(S / K) + (u + 0.5 * sigma_imp ** 2) * t) / (sigma_imp * np.sqrt(t))
    d2 = d1 - (sigma_imp * np.sqrt(t))
    
    n_d1 = norm.cdf(d1)
    n_d2 = norm.cdf(d2)
    
    if cp == 'call':
        price = S * n_d1 - K * np.exp(-u * t) * n_d2
    elif cp == 'put':
        price = K * np.exp(-u * t) * (1 - n_d2) - S * (1 - n_d1)
        
    return price

In [28]:
def Newton_Raphson(S, K, t, initial_guess, u, cp, target_price):
    tolerance = 1e-20
    diff = 1e10
    sigma_imp = initial_guess
    step_size = 0.00000001
    count = 0
    while abs(diff) > tolerance:
        count += 1
        derivative_bs_price_cal = (bs_price_cal(S, K, t, sigma_imp + step_size, u, cp) - bs_price_cal(S, K, t, sigma_imp - step_size, u, cp)) / (2 * step_size)
        sigma_imp = sigma_imp - (bs_price_cal(S, K, t, sigma_imp, u, cp) - target_price)/ derivative_bs_price_cal
        if sigma_imp < 0:
            break
        else:
            diff = bs_price_cal(S, K, t, sigma_imp, u, cp) - target_price

    return sigma_imp, count

In [29]:
bs_price_cal(100, 100, 1, 0.2, 0.02, 'call')

8.916037278572539

In [30]:
Newton_Raphson(100, 100, 1, 0.2, 0.02, 'call', 10)

(0.227723031570633, 3)

In [ ]:
0.27842040930050715

In [17]:
class BS:
    
    """
    This is a class for Options contract for pricing European options on stocks/index without dividends.
    
    Attributes: 
        spot          : int or float
        strike        : int or float 
        rate          : float
        dte           : int or float [days to expiration in number of years]
        volatility    : float
        callprice     : int or float [default None]
        putprice      : int or float [default None]
    """    
    
    def __init__(self, spot, strike, rate, dte, volatility, callprice=None, putprice=None):
        
        # Spot Price
        self.spot = spot
        
        # Option Strike
        self.strike = strike
        
        # Interest Rate
        self.rate = rate
        
        # Days To Expiration
        self.dte = dte
        
        # Volatlity
        self.volatility = volatility
        
        # Callprice # mkt price
        self.callprice = callprice
        
        # Putprice # mkt price
        self.putprice = putprice
            
        # Utility 
        self._a_ = self.volatility * self.dte**0.5
        
        if self.strike == 0:
            raise ZeroDivisionError('The strike price cannot be zero')
        else:
            self._d1_ = (log(self.spot / self.strike) + \
                     (self.rate + (self.volatility**2) / 2) * self.dte) / self._a_
        
        self._d2_ = self._d1_ - self._a_
        
        self._b_ = e**-(self.rate * self.dte)
        
        
        # The __dict__ attribute
        '''
        Contains all the attributes defined for the object itself. It maps the attribute name to its value.
        '''
        for i in ['callPrice', 'putPrice', 'callDelta', 'putDelta', 'callTheta', 'putTheta', \
                  'callRho', 'putRho', 'vega', 'gamma', 'impvol']:
            self.__dict__[i] = None
        
        [self.callPrice, self.putPrice] = self._price()
        [self.callDelta, self.putDelta] = self._delta()
        [self.callTheta, self.putTheta] = self._theta()
        [self.callRho, self.putRho] = self._rho()
        self.vega = self._vega()
        self.gamma = self._gamma()
        self.impvol = self._impvol()
    
    # Option Price
    def _price(self):
        '''Returns the option price: [Call price, Put price]'''

        if self.volatility == 0 or self.dte == 0:
            call = maximum(0.0, self.spot - self.strike)
            put = maximum(0.0, self.strike - self.spot)
        else:
            call = self.spot * norm.cdf(self._d1_) - self.strike * e**(-self.rate * \
                                                                       self.dte) * norm.cdf(self._d2_)

            put = self.strike * e**(-self.rate * self.dte) * norm.cdf(-self._d2_) - \
                                                                        self.spot * norm.cdf(-self._d1_)
        return [call, put]

    # Option Delta
    def _delta(self):
        '''Returns the option delta: [Call delta, Put delta]'''

        if self.volatility == 0 or self.dte == 0:
            call = 1.0 if self.spot > self.strike else 0.0
            put = -1.0 if self.spot < self.strike else 0.0
        else:
            call = norm.cdf(self._d1_)
            put = -norm.cdf(-self._d1_)
        return [call, put]

    # Option Gamma
    def _gamma(self):
        '''Returns the option gamma'''
        return norm.pdf(self._d1_) / (self.spot * self._a_)

    # Option Vega
    def _vega(self):
        '''Returns the option vega'''
        if self.volatility == 0 or self.dte == 0:
            return 0.0
        else:
            return self.spot * norm.pdf(self._d1_) * self.dte**0.5 / 100

    # Option Theta
    def _theta(self):
        '''Returns the option theta: [Call theta, Put theta]'''
        call = -self.spot * norm.pdf(self._d1_) * self.volatility / (2 * self.dte**0.5) - self.rate * self.strike * self._b_ * norm.cdf(self._d2_)

        put = -self.spot * norm.pdf(self._d1_) * self.volatility / (2 * self.dte**0.5) + self.rate * self.strike * self._b_ * norm.cdf(-self._d2_)
        return [call / 365, put / 365]

    # Option Rho
    def _rho(self):
        '''Returns the option rho: [Call rho, Put rho]'''
        call = self.strike * self.dte * self._b_ * norm.cdf(self._d2_) / 100
        put = -self.strike * self.dte * self._b_ * norm.cdf(-self._d2_) / 100

        return [call, put]
    
    # Option Implied Volatility
    def _impvol(self):
        '''Returns the option implied volatility'''
        if (self.callprice or self.putprice) is None:
            return self.volatility
        else:
            def f(sigma):
                option = BS(self.spot,self.strike,self.rate,self.dte,sigma)
                if self.callprice:
                    return option.callPrice - self.callprice # f(x) = BS_Call - MarketPrice
                if self.putprice and not self.callprice:
                    return option.putPrice - self.putprice

            return maximum(1e-5, fsolve(f, 0.2)[0])

In [22]:
def newton_iv(className, spot, strike, rate, dte, volatility, callprice=None, putprice=None):
    
    x0 = 1 # initial guess
    h = 0.001
    tolerance = 1e-7
    epsilon = 1e-14 # some kind of error or floor
    
    maxiter = 200
    
    if callprice:
        # f(x) = Black Scholes Call price - Market Price
        f = lambda x: eval(className)(spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).putPrice - putprice
        
    for i in range(maxiter):
        y = f(x0)
        yprime = (f(x0+h) - f(x0-h))/(2*h) # central difference
        
        if abs(yprime)<epsilon:
            break # this is critial, because volatility cannot be negative
        x1 = x0 - y/yprime
        
        if (abs(x1-x0) <= tolerance*abs(x1)):
            break
        x0=x1
        
    return x1, i

In [23]:
newton_iv('BS',100,100,0.02,1,0.2,callprice=10)

(0.22772303157063303, 3)